<a href="https://colab.research.google.com/github/Ananya63/Project/blob/main/Copy_of_MLCAI_1_3V_running_Including_Random_Mutation_and_Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
import json

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#file_path = '/content/drive/MyDrive/Codon Usage/codon_usage.csv'
file_path ='/content/codon_usage.csv'
df = pd.read_csv(file_path)

In [ ]:
df = df.drop(columns=['Kingdom','DNAtype','SpeciesID','Ncodons'])

In [ ]:
df.shape

(49, 65)

In [ ]:
df = df[(df != 0).all(axis=1)]
#df.replace(0.0, 0.00001, inplace=True)
'''
df.iloc[:, 1:] = df.iloc[:, 1:].replace(0, np.nan)
df = df.dropna()
'''

'\ndf.iloc[:, 1:] = df.iloc[:, 1:].replace(0, np.nan)\ndf = df.dropna()\n'

In [ ]:
df = df.sample(frac=0.4)
#df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

(12, 65)

In [ ]:
def calculate_cai(sequence, codon_usage):
    relative_adaptiveness = {}
    frequencies_numeric = {codon: float(freq) for codon, freq in codon_usage.items()}

    # Normalize codon frequencies
    max_frequency = max(frequencies_numeric.values())
    frequencies_normalized = {codon: freq / max_frequency for codon, freq in frequencies_numeric.items()}

    # Calculate relative adaptiveness
    for codon, freq in frequencies_normalized.items():
        if freq != 0:
            relative_adaptiveness[codon] = np.log(1 + freq)
        else:
            relative_adaptiveness[codon] = 0  # Set rare codons' adaptiveness to 0 or a small value

    # Calculate CAI
    codons = [sequence[i:i+3] for i in range(0, len(sequence), 3)]
    L = len(codons)
    sum_ln_wc = sum(np.log(relative_adaptiveness.get(codon, 1)) for codon in codons)  # Sum of log(relative adaptiveness)
    cai = np.exp(1 / L * sum_ln_wc)

    return cai

In [ ]:
class DNASequenceGenerator:
    def __init__(self, codon_frequencies):
        self.codon_frequencies = codon_frequencies

    def generate_random_dna_sequence(self, length):
        codons = list(self.codon_frequencies.keys())
        return ''.join(random.choice(codons) for _ in range(length))

    def random_mutation(self, sequence):
        position_to_mutate = random.randint(0, len(sequence) - 3)
        new_sequence = np.array(list(sequence), dtype='U1')  # 'U1' dtype for Unicode characters of length 1
        new_codon = random.choice(list(self.codon_frequencies.keys()))
        new_sequence[position_to_mutate:position_to_mutate + 3] = list(new_codon)
        return ''.join(new_sequence)

    def crossover(self, parent1, parent2):
        # Check for empty sequences
        if len(parent1) == 0 or len(parent2) == 0:
            return '', ''

        # Perform a two-point crossover between two parents
        crossover_points = sorted(random.sample(range(len(parent1)), 2))

        # Use NumPy arrays for easier slicing
        np_parent1 = np.array(list(parent1))
        np_parent2 = np.array(list(parent2))

        child1 = np.concatenate([np_parent1[:crossover_points[0]], np_parent2[crossover_points[0]:crossover_points[1]], np_parent1[crossover_points[1]:]])
        child2 = np.concatenate([np_parent2[:crossover_points[0]], np_parent1[crossover_points[0]:crossover_points[1]], np_parent2[crossover_points[1]:]])

        return ''.join(child1), ''.join(child2)

    def generate_multiple_sequences(self, num_sequences, sequence_length):
        sequences = [self.generate_random_dna_sequence(sequence_length) for _ in range(num_sequences)]
        mutated_sequences = [self.random_mutation(seq) for seq in sequences]

        # Perform crossover to generate additional sequences
        for _ in range(num_sequences // 2):
            idx1, idx2 = random.sample(range(num_sequences), 2)
            child1, child2 = self.crossover(mutated_sequences[idx1], mutated_sequences[idx2])
            mutated_sequences.append(child1)
            mutated_sequences.append(child2)

        return mutated_sequences

In [ ]:
class CAIRange:
    def __init__(self):
        self.concentration_lows = {}
        self.concentration_highs = {}

    def analyze_sequences(self, sequences, codon_frequencies):
        cai_values = []
        for sequence in sequences:
            cai = calculate_cai(sequence, codon_frequencies)
            cai_values.append(cai)
        return cai_values

    def perform_clustering(self, cai_values):
        cai_values_reshaped = np.array(cai_values).reshape(-1, 1)
        kmeans = KMeans(n_clusters=1, random_state=42, n_init=10)
        kmeans.fit(cai_values_reshaped)
        center = kmeans.cluster_centers_.flatten()[0]
        std_dev = np.std(cai_values)
        concentration_range_low = center - std_dev/2
        concentration_range_high = center + std_dev/2
        return center, concentration_range_low, concentration_range_high

    def plot_results(self, cai_values, center, concentration_range_low, concentration_range_high, species_name):
        plt.figure(figsize=(10, 2))
        plt.title(f"CAI Values Distribution for {species_name}")
        plt.scatter(cai_values, np.zeros_like(cai_values), color='blue', label='CAI Values', alpha=0.6)
        plt.axvline(x=center, color='red', linestyle='--', label='Cluster Center')
        plt.axvspan(concentration_range_low, concentration_range_high, color='red', alpha=0.3, label='Concentration Range')
        plt.xlabel('CAI Value')
        plt.yticks([])
        plt.legend()
        plt.show()

    def plot_results2(self, cai_values, center, concentration_range_low, concentration_range_high, species_name):
        plt.figure(figsize=(5, 2))
        plt.title(f"CAI Values Distribution for {species_name}")

        # Define the x-axis limits
        x_min = min(center - 0.1 * (concentration_range_high - concentration_range_low), min(cai_values))
        x_max = max(center + 0.1 * (concentration_range_high - concentration_range_low), max(cai_values))

        # Plot concentrated region and a small portion of the outside region
        plt.scatter(cai_values, np.zeros_like(cai_values), color='blue', label='CAI Values', alpha=0.6)
        plt.axvline(x=center, color='red', linestyle='--', label='Cluster Center')
        plt.axvspan(concentration_range_low, concentration_range_high, color='red', alpha=0.3, label='Concentration Range')

        # Set x-axis limits
        plt.xlim(x_min, x_max)

        plt.xlabel('CAI Value')
        plt.yticks([])
        plt.legend()
        plt.show()

    def run(self, species_name, codon_frequencies, num_sequences, sequence_length):
        sequence_generator = DNASequenceGenerator(codon_frequencies)
        sequences = sequence_generator.generate_multiple_sequences(num_sequences, sequence_length)

        cai_values = self.analyze_sequences(sequences, codon_frequencies)
        center, concentration_range_low, concentration_range_high = self.perform_clustering(cai_values)
        self.concentration_lows[species_name] = concentration_range_low
        self.concentration_highs[species_name] = concentration_range_high
        #self.plot_results2(cai_values, center, concentration_range_low, concentration_range_high, species_name)

    def calculate_ranges(self, df, sequence, sequence_length):
        dna_analysis = CAIRange()

        # Lists to store calculated values
        lowcl = []
        highcl = []
        target_cais = []

        # Define margin for the range
        margin = 0.15  # Adjust this value as needed

        for index, row in df.iterrows():
            species_name = row.iloc[0]
            codon_frequencies = {codon: pd.to_numeric(frequency, errors='coerce') for codon, frequency in row.iloc[1:64].items()}
            codon_frequencies = {codon: freq for codon, freq in codon_frequencies.items() if pd.notnull(freq)}
            dna_analysis.run(species_name, codon_frequencies, 50, sequence_length)

            # Calculate values for the current species
            center = (dna_analysis.concentration_lows[species_name] + dna_analysis.concentration_highs[species_name]) / 2
            # Calculate the scaling factor based on the distance from 1
            scaling_factor = 1 / (1 + abs(1 - center))
            # Adjust the target CAI using the scaling factor
            target_cai = center + (1 - center) * scaling_factor

            range_low = target_cai - margin
            range_high = target_cai + margin

            lowcl.append(range_low)
            highcl.append(range_high)
            target_cais.append(target_cai)

        # Add calculated values to 'df'
        df['Range Low'] = lowcl
        df['Range High'] = highcl
        df['Target CAI'] = target_cais


In [ ]:
initial_sequence = "ATCTACGATTACTGGTATAGTAACGTA"
targetCAIgeneration = CAIRange()
targetCAIgeneration.calculate_ranges(df,initial_sequence,len(initial_sequence))

In [ ]:
df.head(30)

,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,...,GAU,GAC,GAA,GAG,UAA,UAG,UGA,Range Low,Range High,Target CAI
44,Guinea pig cytomegalovirus,0.01413,0.02973,0.01040,0.01706,0.00569,0.01868,0.00747,0.02762,0.00650,...,0.02307,0.03721,0.02746,0.03087,0.00065,0.00049,0.00081,0.556926,0.856926,0.706926
21,Bovine herpesvirus 2,0.01499,0.02150,0.00220,0.00800,0.00651,0.02456,0.01146,0.04056,0.00392,...,0.01248,0.04143,0.01153,0.03750,0.00094,0.00039,0.00063,0.499112,0.799112,0.649112
33,Human herpesvirus 6,0.03191,0.01620,0.02215,0.01745,0.01143,0.01155,0.01343,0.01339,0.02413,...,0.03318,0.02081,0.04129,0.01799,0.00089,0.00048,0.00074,0.517554,0.817554,0.667554
32,Murid herpesvirus 1,0.00983,0.02932,0.00685,0.01468,0.00585,0.02202,0.00926,0.02755,0.00588,...,0.02024,0.03233,0.01898,0.02856,0.00109,0.00006,0.00157,0.582324,0.882324,0.732324
26,Equid herpesvirus 1,0.02283,0.01178,0.00584,0.01138,0.00822,0.01865,0.01323,0.03073,0.01056,...,0.01597,0.03541,0.02056,0.03498,0.00119,0.00029,0.00057,0.559056,0.859056,0.709056
12,Variola virus,0.03354,0.01254,0.02725,0.01817,0.01245,0.00509,0.01821,0.00641,0.03814,...,0.05014,0.01553,0.03824,0.01462,0.00222,0.00053,0.00094,0.490800,0.790800,0.640800
13,Raccoonpox virus,0.02749,0.01516,0.02440,0.01618,0.01027,0.00488,0.01952,0.00462,0.04290,...,0.05394,0.01336,0.04624,0.01490,0.00026,0.00103,0.00077,0.474463,0.774463,0.624463
39,Barley mild mosaic virus,0.01904,0.03433,0.00472,0.00795,0.02269,0.03016,0.01233,0.01522,0.01828,...,0.02207,0.03223,0.03023,0.02786,0.00021,0.00045,0.00003,0.569198,0.869198,0.719198
16,Rabbit fibroma virus,0.02889,0.01874,0.02946,0.01972,0.00753,0.00836,0.01454,0.01130,0.02345,...,0.03168,0.02916,0.03534,0.01997,0.00232,0.00055,0.00043,0.519362,0.819362,0.669362
31,Snakehead rhabdovirus,0.01486,0.01982,0.00845,0.01632,0.01778,0.02594,0.01545,0.02477,0.01807,...,0.02215,0.03119,0.03031,0.04459,0.00029,0.00029,0.00117,0.524324,0.824324,0.674324


In [ ]:
# Hypothetical environment setup
class DNAOptimizationEnv:
    def __init__(self, sequence, codon_usage, target_cai):
        self.original_sequence = sequence  # Keep the original sequence
        self.codon_usage = codon_usage
        self.target_cai = target_cai
        self.max_length = len(sequence) // 3  # Number of codons
        self.current_step = 0
        self.sequence = [sequence[i:i+3] for i in range(0, len(sequence), 3)]  # Convert to codons

    def reset(self):
        self.current_step = 0
        self.sequence = [self.original_sequence[i:i+3] for i in range(0, len(self.original_sequence), 3)]
        return self.get_state()

    def step(self, action):
        codon = list(self.codon_usage.keys())[action]
        self.sequence[self.current_step] = codon
        self.current_step += 1

        done = self.current_step >= self.max_length
        reward = self.calculate_reward()
        return self.get_state(), reward, done, {}

    def get_state(self):
        # One-hot encode the sequence of codons
        one_hot_sequence = np.zeros((self.max_length, len(self.codon_usage)))
        for i, codon in enumerate(self.sequence):
            if codon in self.codon_usage:  # Ensure codon is valid
                index = list(self.codon_usage.keys()).index(codon)
                one_hot_sequence[i, index] = 1
            else:
                print(f"Invalid codon: {codon}")  # Debugging aid
        return one_hot_sequence.flatten()

    def calculate_reward(self):
        # Simplified reward function, should be replaced with actual CAI calculation logic
        seq = ''.join(self.sequence)
        current_cai = calculate_cai(seq, self.codon_usage)  # Placeholder for CAI calculation
        reward = -abs(current_cai - self.target_cai)
        return reward

In [ ]:
import torch.nn.functional as F

class PolicyNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(PolicyNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # Increase capacity
        self.fc2 = nn.Linear(512, 256)         # Additional hidden layer
        self.fc3 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(0.2)         # Dropout for regularization

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)                    # Apply dropout
        x = F.relu(self.fc2(x))                # Additional hidden layer
        x = self.fc3(x)
        return F.softmax(x, dim=-1)


In [ ]:
def train(env, policy_model, optimizer, episodes=1000):
    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            action_probs = policy_model(state_tensor)
            action = torch.multinomial(action_probs, 1).item()
            new_state, reward, done, _ = env.step(action)

            # Calculate CAI for the new sequence
            seq = ''.join(env.sequence)
            current_cai = calculate_cai(seq, env.codon_usage)

            # Update reward based on CAI
            reward = env.calculate_reward()

            loss = -torch.log(action_probs[0, action]) * reward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state = new_state

In [ ]:
def predict(env, policy_model):
    state = env.reset()
    done = False
    optimized_sequence = []

    while not done:
        # Convert state to tensor and ensure correct shape
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        # Get action probabilities
        action_probs = policy_model(state_tensor)
        # Sample action (argmax to get the most probable action)
        action = torch.argmax(action_probs).item()
        # Take a step in the environment
        new_state, _, done, _ = env.step(action)

        codon = list(env.codon_usage.keys())[action]
        optimized_sequence.append(codon)

        state = new_state

    return optimized_sequence

In [ ]:
def drive_optimization_for_organism(organism_name, sequence, df, episodes=1000):

    sequence = sequence.replace('T', 'U')

    # Find the row corresponding to the given organism name
    organism_row = df[df['SpeciesName'] == organism_name].iloc[0]

    # Extract codon usage table and target CAI for the organism
    codon_usage = {codon: organism_row[codon] for codon in organism_row.index[1:64]}
    target_cai = organism_row['Target CAI']
    print("Target CAI = ",target_cai)
    range_low = organism_row['Range Low']
    range_high = organism_row['Range High']

    # Initialize the environment with the sequence, codon usage table, and target CAI
    env = DNAOptimizationEnv(sequence, codon_usage, target_cai)

    # Determine input_size for the policy network (number of codons * length of one-hot encoded vector)
    input_size = len(sequence) // 3 * len(codon_usage)
    # Determine output_size for the policy network (number of unique codons)
    output_size = len(codon_usage)

    # Initialize the policy network
    policy_model = PolicyNN(input_size, output_size)

    # Initialize the optimizer
    optimizer = optim.Adam(policy_model.parameters(), lr=0.001)

    optimized_cai = float('inf')  # Initialize optimized CAI to a high value

    while not (range_low <= optimized_cai <= range_high):
        # Train the model
        train(env, policy_model, optimizer, episodes)

        # Predict the optimized sequence using the trained model
        optimized_sequence = predict(env, policy_model)

        # Convert the optimized sequence from codons to a string for printing
        optimized_sequence_str = ''.join(optimized_sequence)

        # Recalculate CAI for the optimized sequence
        # Note: Ensure calculate_cai is compatible with your sequence format and returns CAI directly
        optimized_cai = calculate_cai(optimized_sequence_str, codon_usage)

    return optimized_sequence_str, optimized_cai, range_high, range_low


In [ ]:
def calculate_gc_content(sequence):
  gc_count = sequence.count('G') + sequence.count('C')
  gc_content = gc_count / len(sequence)
  return gc_content

In [ ]:
#organism_name = "Papilio dardanus"
organism_name = df['SpeciesName'].sample(n=1).iloc[0]
#organism_name = input()
initial_sequence = "ATCTACGATTACTGGTATAGTAACGTA"
optimized_sequence,optimized_cai,range_high,range_low = drive_optimization_for_organism(organism_name, initial_sequence, df)
optimized_sequence_str = ''.join(optimized_sequence)
print("Optimized DNA Sequence for", organism_name, ":", optimized_sequence_str)
print("Corresponding CAI Value: {:.2f}".format(optimized_cai))
print("Desired Range: {:.2f} - {:.2f}".format(range_low,range_high))

Target CAI =  0.6141673128753683
Optimized DNA Sequence for Human herpesvirus 1 : CUGCUGCUGCUGCUGCUGCUGCUGCUG
Corresponding CAI Value: 0.57
Desired Range: 0.46 - 0.76


In [ ]:
#organism_name = "Papilio dardanus"
#organism_name = df['SpeciesName'].sample(n=1).iloc[0]
organism_name = input()
original_gc_content = calculate_gc_content(initial_sequence)
print("GC Content of the input sequence : {:.2f}".format(original_gc_content*100))
optimized_sequence,optimized_cai,range_high,range_low = drive_optimization_for_organism(organism_name, initial_sequence, df)
optimized_sequence_str = ''.join(optimized_sequence)
now_gc_content = calculate_gc_content(optimized_sequence_str)
print("Optimized DNA Sequence for", organism_name, ":", optimized_sequence_str)
print("Corresponding CAI Value: {:.2f}".format(optimized_cai))
print("Desired Range: {:.2f} - {:.2f}".format(range_low,range_high))
print("GC Content of the optimized sequence : {:.2f}".format(now_gc_content*100))
print("Increase in GC content : {:.2f}%".format(((now_gc_content-original_gc_content)/original_gc_content)*100) )

GC Content of the input sequence : 33.33
Target CAI =  0.6734213197617376


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error

# Initialize lists to store true and predicted values
true_labels = []
predicted_labels = []
optimized_cai_values = []
range_lows = []
range_highs = []

# Function to calculate the performance metrics
def calculate_performance_metrics(df, initial_sequence, episodes=1000):
    for index, row in df.iterrows():
        organism_name = row['SpeciesName']
        optimized_sequence, optimized_cai, range_high, range_low = drive_optimization_for_organism(organism_name, initial_sequence, df, episodes)

        # True label: 1 if optimized CAI falls within the range, else 0
        true_label = 1 if range_low <= optimized_cai <= range_high else 0
        true_labels.append(true_label)

        # Predicted label: 1 if optimized CAI falls within the range, else 0
        predicted_label = 1 if range_low <= optimized_cai <= range_high else 0
        predicted_labels.append(predicted_label)

        # Store optimized CAI and range values
        optimized_cai_values.append(optimized_cai)
        range_lows.append(range_low)
        range_highs.append(range_high)

    # Calculate accuracy, precision, recall, and mean squared error
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    mse = mean_squared_error(true_labels, predicted_labels)

    # Calculate the average optimized CAI, range low, and range high
    avg_optimized_cai = sum(optimized_cai_values) / len(optimized_cai_values)
    avg_range_low = sum(range_lows) / len(range_lows)
    avg_range_high = sum(range_highs) / len(range_highs)

    return accuracy, precision, recall, mse, avg_optimized_cai, avg_range_low, avg_range_high

# Usage example
initial_sequence = "ATCTACGATTACTGGTATAGTAACGTA"
accuracy, precision, recall, mse, avg_optimized_cai, avg_range_low, avg_range_high = calculate_performance_metrics(df, initial_sequence)

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("Mean Squared Error: {:.2f}".format(mse))
print("Average Optimized CAI: {:.2f}".format(avg_optimized_cai))
print("Average Range Low: {:.2f}".format(avg_range_low))
print("Average Range High: {:.2f}".format(avg_range_high))


Target CAI =  0.7069262792808177
Target CAI =  0.6491122156808358
Target CAI =  0.6675535749770296
Target CAI =  0.7323237315281632
Target CAI =  0.7090562714842856
Target CAI =  0.6407998666205246
Target CAI =  0.62446272500213
Target CAI =  0.7191976214773514


Mycobacterium chelonae

Target CAI :  0.19885163827702967

Corresponding CAI Value: 0.22

Desired Range: 0.18 - 0.22